In [43]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
# from pinecone import Pinecone

In [4]:
PINECONE_API_KEY = "de52f7a8-84a7-4dc0-bf34-a98394867b09"
PINECONE_API_ENV = "gcp-starter"

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data/")


In [11]:
#CREATE TEXT CHUNKS
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)


    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)
len(text_chunks)

7020

In [13]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embeddings = download_hugging_face_embeddings()

c:\a\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
PineconeVectorStore(index_name="medical-chatbot",embedding=embeddings)

vectorstore_from_docs = PineconeVectorStore.from_documents(
    text_chunks,
    index_name="medical-chatbot",
    embedding=embeddings
)

In [48]:
index_name="medical-chatbot"

In [50]:
from pinecone import Pinecone
pinecone = Pinecone(api_key=PINECONE_API_KEY)
docsearch = pinecone.Index("medical-chatbot",  
                             host="https://medical-chatbot-uopezn2.svc.aped-4627-b74a.pinecone.io")

In [58]:
query = "What are Allergies"

# Assuming 'embeddings.embed_query' generates the embedding for the query
query_embedding = embeddings.embed_query(query)
print(query_embedding)
# Use keyword arguments for the query method
results = docsearch.query(vector=query_embedding, top_k=3)

print(results)


[0.0181479062885046, -0.007762155495584011, 0.016550743952393532, -0.02258746139705181, 0.08301930129528046, 0.04213043674826622, 0.047391340136528015, 0.009959175251424313, -0.03739221394062042, 0.07155650854110718, 0.03732006996870041, -0.06228751689195633, 0.017921268939971924, 0.014870434068143368, -0.03633669391274452, 0.08066541701555252, -0.03342955559492111, -0.04579091817140579, -0.0021369094029068947, -0.05767637863755226, -0.014061135239899158, 0.09954631328582764, -0.008254189975559711, -0.005863500293344259, -0.051764920353889465, 0.06957108527421951, -0.037494052201509476, -0.09890787303447723, -0.05588379502296448, -0.03587564453482628, -0.04290857911109924, -0.02302694506943226, 0.0023226344492286444, 0.05676264315843582, -0.04027227684855461, 0.0024839621037244797, -0.08057361096143723, 0.020133905112743378, 0.02193741872906685, 0.007606524974107742, -0.026822227984666824, 0.02393314242362976, -0.06894033402204514, -0.037084147334098816, -0.07588567584753036, 0.0623273

In [64]:
text_chunks[0].page_content

'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'

In [70]:
from langchain.vectorstores import Chroma
persist_directory="db"

vectordb=Chroma.from_documents(documents=text_chunks,embedding=embeddings,persist_directory=persist_directory)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\a\envs\mchatbot\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\aashutosh kumar\AppData\Local\Temp\ipykernel_21784\1317682600.py", line 4, in <module>
    vectordb=Chroma.from_documents(documents=text_chunks,embedding=embeddings,persist_directory=persist_directory)
  File "c:\a\envs\mchatbot\lib\site-packages\langchain\vectorstores\chroma.py", line 462, in from_documents
  File "c:\a\envs\mchatbot\lib\site-packages\langchain\vectorstores\chroma.py", line 423, in from_texts
  File "c:\a\envs\mchatbot\lib\site-packages\langchain\vectorstores\chroma.py", line 90, in __init__
  File "c:\a\envs\mchatbot\lib\site-packages\chromadb\__init__.py", line 334, in Client
    return ClientCreator(tenant=tenant, database=database, settings=settings)
  File "c:\a\envs\mchatbot\lib\site-packages\chromadb\api\client.py", line 55, in __init__
    super().__init_